In [1]:
import torch
from totensor import ToTensor
from PIL import Image
from torchvision import get_image_backend
import json
import os
import math
import copy

# Utilities

In [2]:
def video_loader(video_dir_path,frame_indices):
    video = []
    for i in frame_indices:
        image_path = os.path.join(video_dir_path,'image_{:05d}.jpg'.format(i))
        if os.path.exists(image_path):
            with open(image_path,'rb') as f:
                with Image.open(f) as img:
                    video.append(img.convert('RGB'))
        else:
            return video
    return video
        
def load_value_file(file_path):
    with open(file_path, 'r') as input_file:
        value = float(input_file.read().rstrip('\n\r'))

    return value


# Make data from video

In [3]:
with open('activity_net.v1-3.min.json','r') as data_file:
    dataa =json.load(data_file)


In [ ]:
for key,value in dataa['database'].items():
    print(key)
    print(value['annotations'][0])
    break

In [ ]:
dataa['taxonomy']

In [ ]:
with open('3drKinetics.json','r') as data_file:
    data = json.load(data_file)

In [ ]:
for key,value in data['database'].items():
    print(key)
    print(value['anntoation'])
    break

In [ ]:
data['labels']

In [10]:
def make_dataset(root_path, annotation_path, subset,
                 n_samples_for_each_video, sample_duration,dataset_name ='activitynet'):
    
    # Load annotation data
    with open(annotation_path,'r') as data_file:
        data = json.load(data_file)

    # get video names and annotations
    video_names = []
    annotations = []

    if dataset_name =='activitynet':

        for key, value in data['database'].items():
            this_subset = value['subset']
            if this_subset == subset:
                if subset == 'testing':
                    video_names.append('v_{}'.format(key))
                else:
                    video_names.append('v_{}'.format(key))
                    annotations.append(value['annotations'])
        
        class_names = []
        index = 0
        for node1 in data['taxonomy']:
            is_leaf = True
            for node2 in data['taxonomy']:
                if node2['parentId'] == node1['nodeId']:
                    is_leaf = False
                    break
            if is_leaf:
                class_names.append(node1['nodeName'])

        class_to_idx = {}

        for i, class_name in enumerate(class_names):
            class_to_idx[class_name] = i


    elif dataset_name =='kinetics':
            
        for key, value in data['database'].items():
            this_subset = value['subset']
            if this_subset == subset:
                if subset == 'test':
                    video_names.append('test/{}'.format(key))
                else:

                    label = value['annotations']['label']
                    video_names.append('{}/{}'.format(label, key))
                    annotations.append(value['annotations'])
                    #print('{}/{}'.format(label,key),value['annotations'])

    
        # compute class to label ids 
        class_to_idx ={}
        index = 0
        for class_label in data['labels']:
            class_to_idx[class_label] = index
            index +=1

        
    # compute label to class ids
    idx_to_class ={}
    for name,label in class_to_idx.items():
        idx_to_class[label] = name
        
    dataset = []
    for i in range(len(video_names)):
        if i % 1000 == 0:
            print('dataset loading [{}/{}]'.format(i, len(video_names)))

        video_path = os.path.join(root_path, video_names[i])
        if not os.path.exists(video_path):
            continue
            
            
        #################

        file_names = os.listdir(video_path)
        image_file_names = [x for x in file_names if 'image' in x]
        image_file_names.sort(reverse=True)
        n_frames = int(image_file_names[0][6:11])

 #       n_frames_file_path = os.path.join(video_path, 'n_frames')
 #       n_frames = int(load_value_file(n_frames_file_path))
        if n_frames <= 0:
            continue

        for annotation in annotations[i]:
            
            if dataset_name == 'activitynet':
                begin_t = 1 # math.ceil(annotation['segment'][0] * fps)
                end_t = n_frames #math.ceil(annotation['segment'][1] * fps)
                video_id = video_names[i][2:]
                label = class_to_idx[annotation['label']]
            elif dataset_name == 'kinetics':
                begin_t = 1
                end_t = n_frames
                video_id = video_names[i][:-14].split('/')[1]
                label = class_to_idx[annotation['label']]

            sample = {
                'video': video_path,
                'segment': [begin_t, end_t],
                'n_frames': n_frames,
                'video_id': video_id
            }
            if len(annotations) != 0:
                sample['label'] = class_to_idx[annotation['label']]
            else:
                sample['label'] = -1



            if n_samples_for_each_video == 1:
                sample['frame_indices'] = list(range(1, n_frames + 1))
                dataset.append(sample)
            else:
                if n_samples_for_each_video > 1:
                    step = max(1,
                               math.ceil((n_frames - 1 - sample_duration) /
                                         (n_samples_for_each_video - 1)))
                    print(n_frames,step)
                else:
                    step = sample_duration
                for j in range(1, n_frames, step):
                    sample_j = copy.deepcopy(sample)
                    sample_j['frame_indices'] = list(
                        range(j, min(n_frames + 1, j + sample_duration)))
                    dataset.append(sample_j)

    return dataset, idx_to_class

# Kinetics Dataset Class

In [8]:
class Kinetics(torch.utils.data.Dataset):
    
    def __init__(self,
                root_path,
                annotation_path,
                subset,
                n_samples_for_each_video = 1,
                sample_duration = 16,
                get_loader = video_loader):
        self.data,self.class_names = make_dataset(
            root_path,annotation_path,subset,
            n_samples_for_each_video,sample_duration)
        self.loader = get_loader
        
    def __getitem__(self,index):
        
        path = self.data[index]['video']
        
        frame_indices = self.data[index]['frame_indices']
        
        clip = self.loader(path,frame_indices)
        clip = [ToTensor(1)(img) for img in clip]
        clip = torch.stack(clip,0).permute(1,0,2,3)
        
        target = self.data[index]
        
        return clip,target
    def __len__(self):
        return len(self.data)
        
        
            

# Usage 

In [11]:
training_data = Kinetics('ActivityNet_JPG/','activity_net.v1-3.min.json','training')

#training_data = Kinetics('/mnt/hdd/Kinetics_jpg/','3drKinetics.json','training')

dataset loading [0/10024]
dataset loading [1000/10024]
dataset loading [2000/10024]
dataset loading [3000/10024]
dataset loading [4000/10024]
dataset loading [5000/10024]
dataset loading [6000/10024]
dataset loading [7000/10024]
dataset loading [8000/10024]
dataset loading [9000/10024]
dataset loading [10000/10024]


In [12]:
train_loader = torch.utils.data.DataLoader(
    training_data,
    batch_size=1,
    shuffle=True,
    num_workers = 4,
    pin_memory=True)

In [13]:
for  (inputs,targets) in train_loader:
    print(inputs)
    print(targets)
    break


( 0  , 0  , 0  ,.,.) = 
    0    0    0  ...     0    0    0
   33   34   43  ...    49   49   49
   31   32   46  ...    81   81   81
      ...         ⋱        ...      
  144  144  144  ...   143  142  142
  144  144  144  ...   142  141  141
  144  144  144  ...   142  141  141

( 0  , 0  , 1  ,.,.) = 
    0    0    0  ...     0    0    0
   32   32   41  ...    51   51   51
   31   32   46  ...    79   79   79
      ...         ⋱        ...      
  143  143  143  ...   143  142  142
  143  143  143  ...   143  142  142
  143  143  143  ...   143  142  142

( 0  , 0  , 2  ,.,.) = 
    0    0    0  ...     0    0    0
   28   27   31  ...    51   51   51
   28   26   31  ...    79   79   79
      ...         ⋱        ...      
  143  143  143  ...   143  142  142
  143  143  143  ...   143  142  142
  143  143  143  ...   143  142  142
           ... 

( 0  , 0  ,1048,.,.) = 
  168  168  168  ...   197  197  197
  168  168  168  ...   195  195  195
  168  168  168  ...   193  193  